https://github.com/huggingface/transformers/blob/main/examples/pytorch/contrastive-image-text/run_clip.py#L224

In [94]:
import os
import datasets

COCO_DIR = "/remote-home/songtianwei/research/diffusion_model_my/data"
ds = datasets.load_dataset("ydshieh/coco_dataset_script", "2017",cache_dir=None,data_dir=COCO_DIR)

Found cached dataset coco_dataset_script (/remote-home/songtianwei/.cache/huggingface/datasets/ydshieh___coco_dataset_script/2017-6f3462d26b47c55a/0.0.0/e033205c0266a54c10be132f9264f2a39dcf893e798f6756d224b1ff5078998f)


  0%|          | 0/3 [00:00<?, ?it/s]

In [95]:
ds

DatasetDict({
    train: Dataset({
        features: ['image_id', 'caption_id', 'caption', 'height', 'width', 'file_name', 'coco_url', 'image_path'],
        num_rows: 591753
    })
    validation: Dataset({
        features: ['image_id', 'caption_id', 'caption', 'height', 'width', 'file_name', 'coco_url', 'image_path'],
        num_rows: 25014
    })
    test: Dataset({
        features: ['image_id', 'caption_id', 'caption', 'height', 'width', 'file_name', 'coco_url', 'image_path'],
        num_rows: 40670
    })
})

In [96]:
train_dataset = ds["train"]

In [97]:
for data in train_dataset:
    print(data)
    break

{'image_id': 203564, 'caption_id': 37, 'caption': 'A bicycle replica with a clock as the front wheel.', 'height': 400, 'width': 400, 'file_name': '000000203564.jpg', 'coco_url': 'http://images.cocodataset.org/train2017/000000203564.jpg', 'image_path': '/remote-home/songtianwei/.cache/huggingface/datasets/downloads/extracted/2eaa9620da3c5978de7bf66708380d3175d70ab7bb370443574b53d85fe1ebf0/train2017/000000203564.jpg'}


In [98]:
column_names = ds["train"].column_names

In [99]:
column_names

['image_id',
 'caption_id',
 'caption',
 'height',
 'width',
 'file_name',
 'coco_url',
 'image_path']

In [100]:
from transformers import (
    VisionTextDualEncoderModel,
    VisionTextDualEncoderProcessor,
    AutoTokenizer,
    AutoImageProcessor
)

model = VisionTextDualEncoderModel.from_vision_text_pretrained(
    "openai/clip-vit-base-patch32", "roberta-base"
)

tokenizer = AutoTokenizer.from_pretrained("roberta-base")
image_processor = AutoImageProcessor.from_pretrained("openai/clip-vit-base-patch32")
processor = VisionTextDualEncoderProcessor(image_processor, tokenizer)

# save the model and processor
# model.save_pretrained("clip-roberta")
# processor.save_pretrained("clip-roberta")

Some weights of RobertaModel were not initialized from the model checkpoint at roberta-base and are newly initialized: ['roberta.pooler.dense.weight', 'roberta.pooler.dense.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
The projection layer and logit scale weights `['visual_projection.weight', 'text_projection.weight', 'logit_scale']` are newly initialized. You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Could not find image processor class in the image processor config or the model config. Loading based on pattern matching with the model's feature extractor configuration.


In [101]:
import logging
import os
import sys
from dataclasses import dataclass, field
from typing import Optional

import torch
from datasets import load_dataset
from PIL import Image
from torchvision.io import ImageReadMode, read_image
from torchvision.transforms import CenterCrop, ConvertImageDtype, Normalize, Resize
from torchvision.transforms.functional import InterpolationMode

import transformers
from transformers import (
    AutoImageProcessor,
    AutoModel,
    AutoTokenizer,
    HfArgumentParser,
    Trainer,
    TrainingArguments,
    set_seed,
)
from transformers.trainer_utils import get_last_checkpoint
from transformers.utils import check_min_version, send_example_telemetry
from transformers.utils.versions import require_version

In [92]:
class Transform(torch.nn.Module):
    def __init__(self, image_size, mean, std):
        super().__init__()
        self.transforms = torch.nn.Sequential(
            Resize([image_size], interpolation=InterpolationMode.BICUBIC),
            CenterCrop(image_size),
            ConvertImageDtype(torch.float),
            Normalize(mean, std),
        )

    @torch.jit.ignore
    def add_noise(self, x, noise_type):
        if noise_type == "random":  
            noise = torch.randn_like(x) * 0.1 
        elif  noise_type == "clip_min_noise":
            noise = torch.clamp(x - 0.1, min=0.0, max=1.0) - x
        else:
            raise ValueError(f"Unknown noise type: {noise_type}")
        
        # limit the noise to l2 norm
        noise = noise / torch.norm(noise, dim=-1, keepdim=True)
        # clip to [0, 1]
        x = x + noise
        x = torch.clamp(x, min=0.0, max=1.0)
    
    def forward(self, x, noise_type=None) -> torch.Tensor:
        """`x` should be an instance of `PIL.Image.Image`"""
        with torch.no_grad():
            x = self.transforms(x)
            
        if noise_type is not None:
            x = self.add_noise(x, noise_type)
        return x

In [45]:
train_dataset = train_dataset.map(
        function=tokenize_captions,
        batched=True,
        desc="Running tokenizer on train dataset",
    )

Running tokenizer on train dataset:   0%|          | 0/591753 [00:00<?, ? examples/s]

In [46]:
train_dataset

Dataset({
    features: ['image_id', 'caption_id', 'caption', 'height', 'width', 'file_name', 'coco_url', 'image_path', 'input_ids', 'attention_mask'],
    num_rows: 591753
})

In [47]:
for data in train_dataset:
    print(data)
    break

{'image_id': 203564, 'caption_id': 37, 'caption': 'A bicycle replica with a clock as the front wheel.', 'height': 400, 'width': 400, 'file_name': '000000203564.jpg', 'coco_url': 'http://images.cocodataset.org/train2017/000000203564.jpg', 'image_path': '/remote-home/songtianwei/.cache/huggingface/datasets/downloads/extracted/2eaa9620da3c5978de7bf66708380d3175d70ab7bb370443574b53d85fe1ebf0/train2017/000000203564.jpg', 'input_ids': [0, 250, 14678, 24633, 19, 10, 6700, 25, 5, 760, 5964, 4, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]}


In [86]:
image_column = 'image_path'

In [87]:
image_transformations = Transform(
        224, image_processor.image_mean, image_processor.image_std
    )
image_transformations = torch.jit.script(image_transformations)

In [88]:
def tokenize_captions(examples):
    caption_column = 'caption'
    captions = list(examples[caption_column])
    text_inputs = tokenizer(captions, max_length=80, padding="max_length", truncation=True)
    examples["input_ids"] = text_inputs.input_ids
    examples["attention_mask"] = text_inputs.attention_mask
    return examples

In [89]:
def transform_images(examples):
    images = [read_image(image_file, mode=ImageReadMode.RGB) for image_file in examples['image_path']]
    examples["pixel_values"] = [image_transformations(image) for image in images]
    return examples

In [90]:
def transform_images(examples):
    return examples

In [82]:
from transformers import AutoImageProcessor

image_processor = AutoImageProcessor.from_pretrained("google/vit-base-patch16-224")

In [83]:
def filter_corrupt_images(examples):
        """remove problematic images"""
        valid_images = []
        for image_file in examples[image_column]:
            try:
                Image.open(image_file)
                valid_images.append(True)
            except Exception:
                valid_images.append(False)
        return valid_images

In [64]:
train_dataset = train_dataset.filter(
            filter_corrupt_images, batched=True, num_proc=1
        )

Filter:   0%|          | 0/591753 [00:00<?, ? examples/s]

In [93]:
train_dataset = train_dataset.map(
        function=tokenize_captions,
        batched=True,
        remove_columns=[col for col in column_names if col != image_column],
        num_proc=1,
        desc="Running tokenizer on train dataset",
    )

Running tokenizer on train dataset:   0%|          | 0/36307 [00:00<?, ? examples/s]

/remote-home/songtianwei/conda/envs/pytorch2/lib/python3.8/site-packages/torchvision/transforms/functional.py:1603: UserWarning: The default value of the antialias parameter of all the resizing transforms (Resize(), RandomResizedCrop(), etc.) will change from None to True in v0.17, in order to be consistent across the PIL and Tensor backends. To suppress this warning, directly pass antialias=True (recommended, future default), antialias=None (current default, which means False for Tensors and True for PIL), or antialias=False (only works on Tensors - PIL will still use antialiasing). This also applies if you are using the inference transforms from the models weights: update the call to weights.transforms(antialias=True).
  warnings.warn(


RuntimeError: The following operation failed in the TorchScript interpreter.
Traceback of TorchScript (most recent call last):
  File "/tmp/ipykernel_1250037/2675770440.py", line 31, in forward
            x = self.transforms(x)
            
        if noise_type:
           ~~~~~~~~~~ <--- HERE
            x = self.add_noise(x, noise_type)
        return x
RuntimeError: Boolean value of Tensor with no values is ambiguous


In [ ]:
train_dataset.set_transform(transform_images)

In [ ]:
for data in train_dataset:
    print(data)
    break